In [1]:
%reload_ext autoreload
%autoreload 2

from mne.utils import set_log_level

import os.path as op
import pandas as pd
from hcp_central.s3_utils import download_from_s3_bucket, upload_to_s3, update_s3_file_tracking, get_aws_credentials,s3_glob
from hcp.io.file_mapping import get_s3_keys_meg, get_s3_keys_anatomy
from joblib import Parallel, delayed

set_log_level("warning")

In [2]:
ls /home/ubuntu/mne-hcp-data/*csv

/home/ubuntu/mne-hcp-data/hcp_restricted_data.csv
/home/ubuntu/mne-hcp-data/hcp_unrestricted_data.csv
/home/ubuntu/mne-hcp-data/motor_sensor_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/motor_source_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/rs_sensor_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/rs_source_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/storym_sensor_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/storym_source_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/unrel_subject_processing_map.csv
/home/ubuntu/mne-hcp-data/wm_sensor_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/wm_source_unrelated_subjects.csv


In [3]:
df = pd.read_csv(op.expanduser('~/mne-hcp-data/unrel_subject_processing_map.csv'))

In [4]:
subjects_rs = df.Subject[df.unrelated_rs_source].tolist()

In [5]:
aws_key, aws_secret = get_aws_credentials(
    op.join(op.expanduser("~"), "mne-hcp-aws", 'aws_hcp_details.csv'))

In [6]:
get_s3_keys_anatomy?

In [6]:
def download_subject(subject):
    hcp_prefix = 's3://hcp-openaccess/HCP_900'

    s3_keys = list()
    max_runs = 3
    run_inds = [0, 1, 2]

    hcp_data_types = [
        'rest',
        'noise_empty_room'
    ]

    hcp_outputs = [
        'bads',
        'ica',
        'raw',
    ]

    # allow for downloading fewer data
#     s3_keys = get_s3_keys_meg(
#         subject,
#         data_types=hcp_data_types,
#         hcp_path_bucket='HCP_900',
#         outputs=hcp_outputs,
#         run_inds=run_inds[:max_runs])
#     s3_keys = [kk for kk in s3_keys if 'config' in kk]

    s3_keys += get_s3_keys_anatomy(
        subject,
        hcp_path_bucket='HCP_900')
    download_from_s3_bucket(
        bucket='hcp-openaccess',
        out_path='/mnt/HCP', key_list=s3_keys, aws_key=aws_key,
        aws_secret=aws_secret,
        prefix='HCP_900')
    return subject, s3_keys

In [7]:
def download_anatomy_for_viz_subject(subject):
    hcp_prefix = 's3://hcp-openaccess/HCP_900'

    s3_keys = list()
    
    s3_keys += get_s3_keys_anatomy(
        subject,
        hcp_path_bucket='HCP_900',  freesurfer_outputs=('surf', 'mri'))
    s3_keys = [kk for kk in s3_keys if 'sphere' in kk or 'inflated' in kk or 'c_ras.mat' in kk or
               'MEG_anatomy_transform' in kk or 'MEG_anatomy_headmodel.mat' in kk]

    download_from_s3_bucket(
        bucket='hcp-openaccess',
        out_path='/mnt/HCP', key_list=s3_keys, aws_key=aws_key,
        aws_secret=aws_secret,
        prefix='HCP_900')
    return subject, s3_keys

In [8]:
rm -rf /mnt/HCP/

In [9]:
import mkl

In [10]:
import multiprocessing

In [11]:
multiprocessing.cpu_count()

8

In [12]:
mkl.get_max_threads()

4

In [13]:
len(subjects_rs)

56

In [15]:
import mne
mne.utils.set_log_level('warning')

In [21]:
%%capture

fnames = Parallel(n_jobs=min(mkl.get_max_threads(), len(subjects_rs)))(
    delayed(download_anatomy_for_viz_subject)(subject) for subject in subjects_rs)

In [125]:
!df -h /mnt/HCP

Filesystem      Size  Used Avail Use% Mounted on
/dev/xvdb        79G   61G   14G  82% /mnt


In [23]:
import hcp

In [24]:
project_path = '/mnt/dynamic-scales'
recordings_path = '/mnt/hcp-meg'
hcp_path = '/mnt/HCP'
subjects_dir = '/mnt/hcp_subjects'

for subject in subjects_rs:
    hcp.make_mne_anatomy(subject=str(subject), subjects_dir=subjects_dir,
                         recordings_path=recordings_path, hcp_path=hcp_path)

In [6]:
aws_key_dyn, aws_secret_dyn = get_aws_credentials(
    op.join(op.expanduser("~"), "mne-hcp-aws", 'aws_details.csv'))

In [7]:
hcp_prefix = 's3://hcp-openaccess/HCP_900'
s3_keys = list()
max_runs = 3
run_inds = [0, 1, 2]
for subject in subjects_rs:
    s3_keys = s3_glob(key_pattern='hcp-meg/%s/rest*raw.fif' % subject, bucket='hcp-meg-data',
         aws_key=aws_key_dyn, aws_secret=aws_secret_dyn, prefix='')
    download_from_s3_bucket(
        bucket='hcp-meg-data',
        out_path='/mnt', key_list=s3_keys,
        aws_key=aws_key_dyn,
        aws_secret=aws_secret_dyn,
        prefix='')
    break

Elapsed time downloading hcp-meg-data from s3 00:00:32


In [8]:
ls /home/ubuntu/mne-hcp-aws/

aws_details.csv  aws_dynamic_scales.csv  aws_hcp_details.csv


In [9]:
aws_key_dyn, aws_secret_dyn = get_aws_credentials(
    op.join(op.expanduser("~"), "mne-hcp-aws", 'aws_dynamic_scales.csv'))

In [28]:
%%bash

rm -rf /mnt/HCP

In [29]:
s3_keys = list()
max_runs = 3
run_inds = [0, 1, 2]
for subject in subjects_rs:
    s3_keys = s3_glob(key_pattern='hcp-meg/%s/psd*' % subject, bucket='dynamic-scales',
         aws_key=aws_key_dyn, aws_secret=aws_secret_dyn, prefix='')
    download_from_s3_bucket(
        bucket='dynamic-scales',
        out_path='/mnt', key_list=s3_keys,
        aws_key=aws_key_dyn,
        aws_secret=aws_secret_dyn,
        prefix='')


Elapsed time downloading dynamic-scales from s3 00:00:09
Elapsed time downloading dynamic-scales from s3 00:00:00
Elapsed time downloading dynamic-scales from s3 00:00:08
Elapsed time downloading dynamic-scales from s3 00:00:08
Elapsed time downloading dynamic-scales from s3 00:00:09
Elapsed time downloading dynamic-scales from s3 00:00:08
Elapsed time downloading dynamic-scales from s3 00:00:08
Elapsed time downloading dynamic-scales from s3 00:00:07
Elapsed time downloading dynamic-scales from s3 00:00:09
Elapsed time downloading dynamic-scales from s3 00:00:08
Elapsed time downloading dynamic-scales from s3 00:00:08
Elapsed time downloading dynamic-scales from s3 00:00:09
Elapsed time downloading dynamic-scales from s3 00:00:09
Elapsed time downloading dynamic-scales from s3 00:00:07
Elapsed time downloading dynamic-scales from s3 00:00:07
Elapsed time downloading dynamic-scales from s3 00:00:09
Elapsed time downloading dynamic-scales from s3 00:00:08
Elapsed time downloading dynami